In [1]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [31]:
meta_dir = '/home/ggorin/count_data/city_of_hope/meta/'
# meta_file = meta_dir+'PSCA_17483_metadata.xlsx'
meta_file = meta_dir + 'IRB17483_metadata.csv'

In [57]:
lib_names = ['UPN375_388_PSCA_PB_S_X5SCR_SE8003_SA157968',
'UPN388_394_PSCA_PB_P_X5SCR_SE8003_SA157965',
'UPN375_394_PSCA_PB_R_X5SCR_SE8003_SA157967',
'UPN388_394_PSCA_PB_Q_X5SCR_SE8003_SA157966',
'UPN375_403_406_PB_G_X5SCR_SE6436_SA71767',
'UPN388_406_PB_I_X5SCR_SE6438_SA71770',
'UPN375_376_388_393_PSCA_CART_X5SCR_SE6080_SA58165',
'UPN375_403_406_PB_H_X5SCR_SE6437_SA71768',
'UPN393_403_406_PB_D_X5SCR_SE6452_SA71980',
'UPN375_376_393_PB_A_X5SCR_SE6452_SA71977',
'UPN376_388_403_PB_C_X5SCR_SE6452_SA71979',
'UPN394_531_555_CART_L_X5SCR_SE7662_SA134623',
'UPN375_388_393_PB_B_X5SCR_SE6452_SA71978',
'UPN376_388_406_PB_E_X5SCR_SE6452_SA71981',
'UPN454_455_504_CART_K_X5SCR_SE7661_SA134622',
'UPN375_388_PSCA_PB_N_X5SCR_SE8003_SA157963',
'UPN376_393_406_PB_F_X5SCR_SE6435_SA71769',
'UPN375_388_PSCA_PB_O_X5SCR_SE8003_SA157964',
'UPN388_394_PSCA_PB_M_X5SCR_SE8003_SA157962']
lib_dict = {}
for x in lib_names:
    y = x.rsplit('_',2)
    lib_dict[ '_'.join((y[1],y[2]))] = x

In [62]:
# lib_dict

In [65]:
df_meta = pd.read_csv(meta_file)

In [66]:
df_meta = df_meta[(df_meta['sequencing_platform']=='NovaSeqS4') & (df_meta['tissue_type']=='PB')]
df_meta['timepoint'] = df_meta['timepoint'].astype(int)

In [67]:
df_meta = df_meta.sort_values(['upn','timepoint'])

In [68]:
df_meta

,upn,timepoint,tissue_type,sample_name,library_id,flowcell_id,sequencing_platform,library_type
24,375,0,PB,PB_A,SE6452_SA71977,HJNN2DSXY,NovaSeqS4,X5SCR
103,375,0,PB,PB_S,SE8003_SA157968,H72NVDSX5,NovaSeqS4,X5SCR
30,375,1,PB,PB_B,SE6452_SA71978,HJNN2DSXY,NovaSeqS4,X5SCR
60,375,14,PB,PB_G,SE6436_SA71767,HKNHWDSXY,NovaSeqS4,X5SCR
82,375,14,PB,PB_N,SE8003_SA157963,H72NVDSX5,NovaSeqS4,X5SCR
66,375,28,PB,PB_H,SE6437_SA71768,HJNN3DSXY,NovaSeqS4,X5SCR
86,375,28,PB,PB_O,SE8003_SA157964,H72NVDSX5,NovaSeqS4,X5SCR
98,375,90,PB,PB_R,SE8003_SA157967,H72NVDSX5,NovaSeqS4,X5SCR
36,376,0,PB,PB_C,SE6452_SA71979,HJNN2DSXY,NovaSeqS4,X5SCR
25,376,1,PB,PB_A,SE6452_SA71977,HJNN2DSXY,NovaSeqS4,X5SCR


Best donors: 388, 375, 394

In [63]:
donor_ids = [388]

In [81]:

for donor in donor_ids:
    print(f'Donor: {donor}')
    df_donor = df_meta[df_meta['upn']==donor]
    t_points = df_donor['timepoint'].unique()
    for t in t_points:
        print(f'\tTime point: {t}')
        df_t = df_donor[df_donor['timepoint'] == t]
        for lib in df_t['library_id']:
            print(f'\t\tLibrary: {lib}')
            lib_meta = meta_dir + lib_dict[lib]+'_donor_ids.csv'
            

Donor: 388
	Time point: -6
		Library: SE8003_SA157962
	Time point: 0
		Library: SE6452_SA71981
		Library: SE8003_SA157968
	Time point: 1
		Library: SE6438_SA71770
	Time point: 14
		Library: SE6452_SA71978
		Library: SE8003_SA157964
	Time point: 28
		Library: SE6452_SA71979
		Library: SE8003_SA157965
	Time point: 90
		Library: SE8003_SA157966
	Time point: 224
		Library: SE8003_SA157963


In [85]:
lib_dict[lib]

'UPN375_388_PSCA_PB_N_X5SCR_SE8003_SA157963'

In [84]:
pd.read_csv(lib_meta)

FileNotFoundError: [Errno 2] No such file or directory: '/home/ggorin/count_data/city_of_hope/meta/UPN375_388_PSCA_PB_N_X5SCR_SE8003_SA157963_donor_ids.csv'